In this notebook we will be doing the pre-processing and modeling steps to build a model which would predict whether a patient has pneumonia or not using chest X-ray images. 
We will be building a convolutional neural network for predicting the patient status. Convolutional neural networks are commonly used for predictions in image based projects. They are used  due to their ability to extract local features,  with the use of kernels in the convolution layers  that are used to convolute the image to create feature maps. Along with convolutional layers there are pooling layers which extract the most important features and reduce the spatial dimensions.  


The resnet model was used in the end and I reprocessed the images numpy array so that it was no long grey scaled and was in RGB channel.

Let us load libraries required. We will be using tensor flow's keras API for modeling the convolutional neural network due to its ease of use. 

In [1]:
import cv2
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.metrics import AUC, Precision, Recall, Accuracy
import sklearn.utils
from sklearn.model_selection import StratifiedKFold



Now we will load the training numpy arrays. We have the normal and pneumonia numpy arrays which we will load separately. We will make the index labels for the pneumonia and normal images. We will follow this by concatenating the numpy arrays and also the index labels for the training numpy arrays

In [11]:
Pneumonia_training=np.load("/Users/mks9338/Documents/Course/Capstone_three/images_pneumonia_training.npy")
Normal_training=np.load("/Users/mks9338/Documents/Course/Capstone_three/images_normal_training.npy")

In [16]:
print(Pneumonia_training.shape)
print(Normal_training.shape)

(3875, 224, 224)
(1341, 224, 224)


In [12]:
pneumonia_label=np.ones(Pneumonia_training.shape[0])
normal_label=np.zeros(Normal_training.shape[0])

In [17]:
training=np.vstack((Pneumonia_training,Normal_training))
training_labels=np.concatenate((pneumonia_label,normal_label))

Will now normalise the pixels to be between 0 and 1 by dividing by 255

In [5]:
training_normalise=training/255

In [16]:
training_normalise.shape

(5216, 224, 224)

We will start with training using a simple CNN model and see how training helps.
We will start with developing three convolutional layers max pooling each convolutional layer. The layers are then flattened and added to the developed 

In [6]:
training_normalise = np.expand_dims(training_normalise, axis=-1)

In [20]:
training_normalise.shape

(5216, 224, 224, 1)

We started with building a simple CNN framework, where layers were added sequentially and parameters based on suggestions available online. We first built a convolution layer with kernel size of 3x3 with a total of 32 kernels. The activation function used in cnn is the non-linear RELU function, which returns the input value if it is positive and 0 otherwise. This helps it overcome the vanishing gradient problem with other functions during backpropogation. We then followed it up with max-pooling to reduce the number of features wth the pol_size (2,2). The next convolutional layer was built with a 3x3 kernel with a total of 64 kernels, increasing the complexity of the convolutional layer with max pooling. The layers were then flattened and then the nodes from the second convolutional layer passed onto a  dense network with 128 neurons and the RELU activation function. The output was passed onto a single neuron which gave a probability score to classify an image as pneumonia or normal (1,0). Sigmoid function is used so that the output is a probability score.
For optimization of weights, biases and learning rate we used the Adam optimizer, which is commonly used in image based cnn networks. Adam optimizer calculates changes based on using a combination of stochastic gradient descent and momentum.We used the binary cross entropy loss function, which is commonly used in classification neural network problems 

In [39]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # For binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[AUC(),Precision(), Recall(),Accuracy()])

# Now you can use `images_reshaped` as the input to the model
model.fit(training_normalise, training_labels, epochs=10)

Epoch 1/10
163/163 [==============================] - 64s 386ms/step - loss: 0.2734 - auc_6: 0.9480 - precision_6: 0.9143 - recall_6: 0.9445 - accuracy: 0.0033
Epoch 2/10
163/163 [==============================] - 64s 392ms/step - loss: 0.0888 - auc_6: 0.9933 - precision_6: 0.9770 - recall_6: 0.9768 - accuracy: 0.0082
Epoch 3/10
163/163 [==============================] - 66s 406ms/step - loss: 0.0546 - auc_6: 0.9969 - precision_6: 0.9861 - recall_6: 0.9855 - accuracy: 0.0326
Epoch 4/10
163/163 [==============================] - 65s 400ms/step - loss: 0.0514 - auc_6: 0.9975 - precision_6: 0.9876 - recall_6: 0.9879 - accuracy: 0.0656
Epoch 5/10
163/163 [==============================] - 64s 396ms/step - loss: 0.0257 - auc_6: 0.9992 - precision_6: 0.9938 - recall_6: 0.9933 - accuracy: 0.1191
Epoch 6/10
163/163 [==============================] - 68s 420ms/step - loss: 0.0161 - auc_6: 0.9998 - precision_6: 0.9964 - recall_6: 0.9959 - accuracy: 0.2170
Epoch 7/10
163/163 [====================

The AUC is perfect , but the accuracy is poor only 39%. In an imbalanced dataset where the number of pneumonia images are muvh higher if the model performs well, you should get a very high accuracy. We need to correct class imbalance and choose another model with more layers because despite imbalance accuracy is very low.

We will deal with class balance by adjusting the weights of the class. The under represented minority class will be given more weight so that if it is misclassified. 

In [40]:
from sklearn.utils import class_weight
class_weights_array = class_weight.compute_class_weight('balanced', classes=np.unique(training_labels),y=training_labels)

class_weights={i: class_weights_array[i] for i in range(len(class_weights_array))}

# Output the class weights
print(class_weights)


{0: 1.9448173005219984, 1: 0.6730322580645162}


We will fit a model with an additional convolutional layer added and adding the class weights.

In [41]:
model1 = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # For binary classification
])

model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=[AUC(),Precision(), Recall(),Accuracy()])

In [46]:
model1.fit(training_normalise, training_labels, epochs=20,class_weight=class_weights)

Epoch 1/20
163/163 [==============================] - 81s 493ms/step - loss: 4.2267e-06 - auc_7: 1.0000 - precision_7: 1.0000 - recall_7: 1.0000 - accuracy: 0.6513
Epoch 2/20
163/163 [==============================] - 83s 510ms/step - loss: 3.0206e-06 - auc_7: 1.0000 - precision_7: 1.0000 - recall_7: 1.0000 - accuracy: 0.6547
Epoch 3/20
163/163 [==============================] - 82s 506ms/step - loss: 2.2973e-06 - auc_7: 1.0000 - precision_7: 1.0000 - recall_7: 1.0000 - accuracy: 0.6597
Epoch 4/20
163/163 [==============================] - 82s 501ms/step - loss: 1.9051e-06 - auc_7: 1.0000 - precision_7: 1.0000 - recall_7: 1.0000 - accuracy: 0.6620
Epoch 5/20
163/163 [==============================] - 78s 479ms/step - loss: 1.5709e-06 - auc_7: 1.0000 - precision_7: 1.0000 - recall_7: 1.0000 - accuracy: 0.6658
Epoch 6/20
163/163 [==============================] - 77s 474ms/step - loss: 1.2805e-06 - auc_7: 1.0000 - precision_7: 1.0000 - recall_7: 1.0000 - accuracy: 0.6678
Epoch 7/20
163/1

The accuracy increases to 0.70 by just adding one more layer and then also correcting for imbalance by ajusting class weights. However adjusting class weights does not help us with dealing the bias associated with class imbalance related accuracy differences

 Rather than correcting imbalance by adjusting class weights, let us try image augmentation in which we increase the number of images for the normal class to balance it with the pneumonia class for class imbalance and use the same model and check accuracy. 

In [9]:
len(Pneumonia_training)-len(Normal_training)

2534

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Normal_training_normalise= np.expand_dims(Normal_training, axis=-1)

# Define an ImageDataGenerator for augmentation
datagen = ImageDataGenerator(
    rotation_range=5,
    zoom_range=0.2
)

# Fit the generator to the minority class images
datagen.fit(Normal_training_normalise)

# Generate augmented images
augmented_images = []
augmented_labels = []

# Number of augmented images to generate
n_augmented = len(Pneumonia_training)-len(Normal_training) # For example, generate 1000 augmented images

for i in range(n_augmented):
    for img_batch, label_batch in datagen.flow(Normal_training_normalise, normal_label, batch_size=1):
        augmented_images.append(img_batch[0])  # add augmented image
        augmented_labels.append(label_batch[0])  # add augmented label
        if len(augmented_images) >= n_augmented:
            break
    if len(augmented_images) >= n_augmented:
        break

augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)


Now that we have augmented normal images and augmented normal labels, we can add them to the training dataset. 

In [23]:
augmented_labels

array([0., 0., 0., ..., 0., 0., 0.])

In [8]:
augmented_images=augmented_images/255
training_augmented=np.vstack((training_normalise,augmented_images))
training_augmented_labels=np.concatenate((training_labels,augmented_labels))

Now let us use the convolutional model again with the augmented images added, we can increase the number of epochs 20

In [45]:
model1.fit(training_augmented,training_augmented_labels, epochs=20)

Epoch 1/20
243/243 [==============================] - 135s 554ms/step - loss: 0.1079 - auc_7: 0.9924 - precision_7: 0.9622 - recall_7: 0.9590 - accuracy: 0.0021
Epoch 2/20
243/243 [==============================] - 125s 514ms/step - loss: 0.0554 - auc_7: 0.9978 - precision_7: 0.9786 - recall_7: 0.9778 - accuracy: 0.0175
Epoch 3/20
243/243 [==============================] - 121s 498ms/step - loss: 0.0349 - auc_7: 0.9992 - precision_7: 0.9891 - recall_7: 0.9868 - accuracy: 0.0514
Epoch 4/20
243/243 [==============================] - 127s 525ms/step - loss: 0.0157 - auc_7: 0.9999 - precision_7: 0.9953 - recall_7: 0.9941 - accuracy: 0.1418
Epoch 5/20
243/243 [==============================] - 132s 545ms/step - loss: 0.0129 - auc_7: 0.9999 - precision_7: 0.9954 - recall_7: 0.9951 - accuracy: 0.2498
Epoch 6/20
243/243 [==============================] - 131s 541ms/step - loss: 0.0134 - auc_7: 0.9998 - precision_7: 0.9956 - recall_7: 0.9959 - accuracy: 0.2105
Epoch 7/20
243/243 [==============

There is a decreased accuracy only acheiving 0.43 by using the image augmentation to improve class imbalance. Accuracy is an accurate metric to consider here as we have balanced classes by having equal number of pneumonia and normal images. We need to add more layers to see if it helps improve accuracy.

In [48]:
model2 = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # For binary classification
])

model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=[AUC(),Precision(), Recall(),Accuracy()])

Does not work well.

In [49]:
model2.fit(training_augmented,training_augmented_labels, epochs=20)

Epoch 1/20
243/243 [==============================] - 151s 618ms/step - loss: 0.2671 - auc_8: 0.9563 - precision_8: 0.8913 - recall_8: 0.8526 - accuracy: 7.7419e-04
Epoch 2/20
243/243 [==============================] - 136s 559ms/step - loss: 0.1283 - auc_8: 0.9890 - precision_8: 0.9547 - recall_8: 0.9515 - accuracy: 1.2903e-04
Epoch 3/20
243/243 [==============================] - 137s 566ms/step - loss: 0.0950 - auc_8: 0.9937 - precision_8: 0.9672 - recall_8: 0.9597 - accuracy: 9.0323e-04
Epoch 4/20
243/243 [==============================] - 135s 557ms/step - loss: 0.0831 - auc_8: 0.9947 - precision_8: 0.9720 - recall_8: 0.9693 - accuracy: 0.0043
Epoch 5/20
243/243 [==============================] - 135s 554ms/step - loss: 0.0708 - auc_8: 0.9965 - precision_8: 0.9759 - recall_8: 0.9698 - accuracy: 0.0075
Epoch 6/20
243/243 [==============================] - 135s 555ms/step - loss: 0.0565 - auc_8: 0.9977 - precision_8: 0.9791 - recall_8: 0.9781 - accuracy: 0.0135
Epoch 7/20
243/243 [==

Does not work actually the accuracy decreases by adding another layer and increasing the number of kernels. Best is to use a pre-trained model sing transfer learning and change its architecture.

We will be using the Resnet50 model, which has been previously used for X ray image classifications. Resnet50 stands for Residual Net and the model is a 50 layer deep model. In a residual network model, residual blocks are introduced in which in case layers are skipped if they reduce the performance of the network, which is a problem encountered with deep layer models. 


In [3]:
from keras.applications import ResNet50V2
resnet50 = ResNet50V2(weights = "imagenet", input_shape = (224,224,3), include_top = False)

In [11]:
resnet50.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                         

Resnet only works for 3 channel images so I need to reprocess my data to remove grey scaling before using a resnet model

In [4]:
training_folder_normal= '/Users/mks9338/Documents/Course/Capstone_three/chest_xray/train/NORMAL/'
training_folder_pneumonia= '/Users/mks9338/Documents/Course/Capstone_three/chest_xray/train/PNEUMONIA/'
image_paths_normal = glob.glob(training_folder_normal + '*.jpeg')
image_paths_pneumonia = glob.glob(training_folder_pneumonia + '*.jpeg')

images_normal_training = []

for path in image_paths_normal:
     image = cv2.imread(path)
     img_resize=cv2.resize(image,(224,224))
     if img_resize is not None:
        images_normal_training.append(img_resize)
     else:
       print(f"Failed to load image: {path}")

images_pneumonia_training = []

for path in image_paths_pneumonia:
     image = cv2.imread(path)
     img_resize=cv2.resize(image,(224,224))
     if img_resize is not None:
        images_pneumonia_training.append(img_resize)
     else:
       print(f"Failed to load image: {path}")


In [5]:
training_new=np.vstack((images_normal_training,images_pneumonia_training))

In [6]:
training_normalise_new=training_new/255

In [7]:
training_normalise_new.shape

(5216, 224, 224, 3)

In [18]:
np.array(images_normal_training).shape

(1341, 224, 224, 3)

In [14]:
normal_label.shape

(1341,)

In [15]:

#normal_label
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define an ImageDataGenerator for augmentation
datagen = ImageDataGenerator(
    rotation_range=5,
    zoom_range=0.2
)
# Fit the generator to the minority class images
datagen.fit(np.array(images_normal_training))

# Generate augmented images
augmented_images_new = []
augmented_labels_new = []
n_augmented = len(Pneumonia_training)-len(Normal_training)
# Number of augmented images to generate

for i in range(n_augmented):
    for img_batch, label_batch in datagen.flow(np.array(images_normal_training), normal_label, batch_size=1):
        augmented_images_new.append(img_batch[0])  # add augmented image
        augmented_labels_new.append(label_batch[0])  # add augmented label
        if len(augmented_images_new) >= n_augmented:
            break
    if len(augmented_images_new) >= n_augmented:
        break

augmented_images_new = np.array(augmented_images_new)
augmented_labels_new = np.array(augmented_labels_new)

In [18]:
augmented_images_new=augmented_images_new/255
training_augmented_new=np.vstack((training_normalise_new,augmented_images_new))
training_augmented_labels_new=np.concatenate((training_labels,augmented_labels_new))

In [20]:
# Freeze all layers of ResNet50

model3 = Sequential()

model3.add(resnet50)

for layer in resnet50.layers:
    layer.trainable = False
    
model3.add(Flatten())

model3.add(Dense(units = 128, activation = "relu"))

model3.add(Dense(units = 1, activation = "sigmoid"))



In [21]:
model3.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [23]:
model3.fit(training_augmented_new,training_augmented_labels_new, epochs=10)

Epoch 1/10


2025-01-21 17:19:38.076546: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


243/243 [==============================] - 204s 834ms/step - loss: 1.0198 - accuracy: 0.7764
Epoch 2/10
243/243 [==============================] - 203s 834ms/step - loss: 0.2684 - accuracy: 0.8739
Epoch 3/10
243/243 [==============================] - 196s 805ms/step - loss: 0.1416 - accuracy: 0.9486
Epoch 4/10
243/243 [==============================] - 196s 806ms/step - loss: 0.0984 - accuracy: 0.9681
Epoch 5/10
243/243 [==============================] - 199s 817ms/step - loss: 0.0598 - accuracy: 0.9863
Epoch 6/10
243/243 [==============================] - 199s 819ms/step - loss: 0.0487 - accuracy: 0.9908
Epoch 7/10
243/243 [==============================] - 200s 822ms/step - loss: 0.0534 - accuracy: 0.9877
Epoch 8/10
243/243 [==============================] - 202s 830ms/step - loss: 0.0602 - accuracy: 0.9819
Epoch 9/10
243/243 [==============================] - 203s 833ms/step - loss: 0.0724 - accuracy: 0.9738
Epoch 10/10
243/243 [==============================] - 202s 830ms/step - lo

Use of the pre-trained model works really well with the use of a resnet 50 model. It acheives an accuracy of 0.9850. Now I need to look at the testing dataset for this. 

The testing data set also needs to be converted to an RGB channel frequency again so that it can be used to predict the training datset\

In [24]:
testing_folder_normal= '/Users/mks9338/Documents/Course/Capstone_three/chest_xray/test/NORMAL/'
testing_folder_pneumonia= '/Users/mks9338/Documents/Course/Capstone_three/chest_xray/test/PNEUMONIA/'
image_paths_normal_test = glob.glob(testing_folder_normal + '*.jpeg')
image_paths_pneumonia_test = glob.glob(testing_folder_pneumonia + '*.jpeg')
print("Number of testing images which are normal", len(image_paths_normal_test))
print("Number of testing images which are pneumonia", len(image_paths_pneumonia_test))

Number of testing images which are normal 234
Number of testing images which are pneumonia 390


In [25]:
images_normal_testing = []

for path in image_paths_normal_test :
     image = cv2.imread(path)
     img_resize=cv2.resize(image,(224,224))
     if img_resize is not None:
        images_normal_testing.append(img_resize)
     else:
       print(f"Failed to load image: {path}")

images_pneumonia_testing = []

for path in image_paths_pneumonia_test:
     image = cv2.imread(path)
     img_resize=cv2.resize(image,(224,224))
     if img_resize is not None:
        images_pneumonia_testing.append(img_resize)
     else:
       print(f"Failed to load image: {path}")

In [26]:
validation_folder_normal= '/Users/mks9338/Documents/Course/Capstone_three/chest_xray/val/NORMAL/'
validation_folder_pneumonia= '/Users/mks9338/Documents/Course/Capstone_three/chest_xray/val/PNEUMONIA/'
image_paths_normal_validation = glob.glob(validation_folder_normal + '*.jpeg')
image_paths_pneumonia_validation = glob.glob(validation_folder_pneumonia + '*.jpeg')


In [28]:
images_normal_validation = []

for path in image_paths_normal_validation :
     image = cv2.imread(path)
     img_resize=cv2.resize(image,(224,224))
     if img_resize is not None:
        images_normal_validation.append(img_resize)
     else:
       print(f"Failed to load image: {path}")

images_pneumonia_validation = []

for path in image_paths_pneumonia_validation:
     image = cv2.imread(path)
     img_resize=cv2.resize(image,(224,224))
     if img_resize is not None:
        images_pneumonia_validation.append(img_resize)
     else:
       print(f"Failed to load image: {path}")

In [29]:
combined_testing_normal_image = np.concatenate((images_normal_testing,images_normal_validation),axis=0)
combined_testing_pneumonia_image = np.concatenate((images_pneumonia_testing,images_pneumonia_validation),axis=0)

In [34]:
pneumonia_label_test=np.ones(combined_testing_normal_image.shape[0])
normal_label_test=np.zeros(combined_testing_pneumonia_image.shape[0])

In [35]:
validation_final=np.vstack((combined_testing_pneumonia_image,combined_testing_normal_image))
validation_labels=np.concatenate((pneumonia_label_test,normal_label_test))

In [36]:
validation_final_normalise=validation_final/255

In [38]:
val_loss, val_accuracy = model3.evaluate(validation_final_normalise,validation_labels)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')

20/20 [==============================] - 16s 765ms/step - loss: 4.0346 - accuracy: 0.3688
Validation Loss: 4.034557819366455
Validation Accuracy: 0.3687500059604645
